In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def sigmoid(z):
    a = 1/(1+np.exp(-z))
    return a

def relu(z):
    a = np.maximum(0, z)
    return a

def leaky_relu(z, aplha=0.01):
    a = np.maximum(aplha*z, z)
    return a

def tanh(z):
    a = (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))
    return a

def softmax(z):
    m = z.shape[-1]
    exp_z = np.exp(z)
    a = np.divide(exp_z, np.sum(exp_z)/m)
    return a

def activate(z, activation):
    if activation == "sigmoid":
        return sigmoid(z)
    elif activation == "relu":
        return relu(z)
    elif activation == "leaky_relu":
        return leaky_relu(z)
    elif activation == "tanh":
        return tanh(z)
    else:
        return softmax(z)

class layer:
    def __init__(self, shape, activation):
        self.W = np.random.randn(*shape)*0.01
        self.b = np.zeros((shape[0], 1))
        self.activation = activation
        
    def forward(self, x):
        self.z = np.dot(self.W, x) + self.b
        self.a = activate(self.z, self.activation)
        return self.a, self.W, self.z, x

def backward_propagation_sigmoid(dal, wl, zl, al_1):
    m = dal.shape[-1]
    al = sigmoid(zl)
    dzl = dal * al * (1 - al)
    dwl = 1/m*np.dot(dzl, al_1.T)
    dbl = 1/m*np.sum(dzl, axis=1, keepdims=True)
    dal_1 = np.dot(wl.T, dzl)
    return dal_1, dwl, dbl

def backward_propagation_relu(dal, wl, zl, al_1):
    m = dal.shape[-1]
    al = relu(zl)
    dal_dzl = zl >= 0
    dal_dzl = dal_dzl.astype("int")
    dzl = dal*dal_dzl
    dwl = 1/m*np.dot(dzl, al_1.T)
    dbl = 1/m*np.sum(dzl, axis=1, keepdims=True)
    dal_1 = np.dot(wl.T, dzl)
    return dal_1, dwl, dbl

def backward_propagation_tanh(dal, wl, zl, al_1):
    m = dal.shape[-1]
    al = tanh(zl)
    dzl = dal*(1-al**2)
    dwl = 1/m*np.dot(dzl, al_1.T)
    dbl = 1/m*np.sum(dzl, axis=1, keepdims=True)
    dal_1 = np.dot(wl.T, dzl)
    return dal_1, dwl, dbl

def backward_propagation(dal, wl, zl, al_1, act):
    if act == "sigmoid":
        return backward_propagation_sigmoid(dal, wl, zl, al_1)
    elif act == "relu":
        return backward_propagation_relu(dal, wl, zl, al_1)
    else:
        return backward_propagation_tanh(dal, wl, zl, al_1)

def select(a, c):
    ac = []
    for i, j in zip(c[0, :], range(c.shape[-1])):
        ac.append(a[i, j])
    ac = np.array(ac).reshape(1, len(ac))
    return ac

def grads_correction(dal_dzl, temp, c):
    for idx, i in zip(c[0, :], range(dal_dzl.shape[-1])):
        dal_dzl[idx, i] = temp[0, i]
    return dal_dzl

def backward_propagation_softmax(dal, wl, zl, al_1, c):
    m = dal.shape[-1]
    al = softmax(zl)
    ac = select(al, c)
    dal_dzl = -al * ac
    temp = ac*(1-ac)
    dal_dzl = grads_correction(dal_dzl, temp, c)
    dzl = dal*dal_dzl
    dwl = 1/m*np.dot(dzl, al_1.T)
    dbl = 1/m*np.sum(dzl, axis=1, keepdims=True)
    dal_1 = np.dot(wl.T, dzl)
    return dal_1, dwl, dbl

def one_hot(num_classes, y):
    zero_array = np.zeros((num_classes, y.shape[-1]))
    for y_, i in zip(y[0, :], range(y.shape[-1])):
        zero_array[y_, i] = 1
    return zero_array

# create batches of data
def batchify(X, Y, batch_size):
    m = X.shape[-1]
    for idx in range(0, m, batch_size):
        yield(X[:, idx:min(idx + batch_size, m)], Y[:, idx:min(idx + batch_size, m)])

class model:
    def __init__(self):
        self.layers = []
    
    def forward_prop(self, x):
        cache = {}
        al = x
        for layer, l in zip(self.layers, range(1, len(self.layers)+1)):
            al, wl, zl, al_1 = layer.forward(al)
            cache["l"+str(l)] = [wl, zl, al_1]
        return al, cache
    
    def backward_prop(self, y_hat, y, cache):
        grads = {}
        y_onehot = one_hot(y_hat.shape[0], y)
        m = y.shape[-1]
        dzl = np.copy(y_hat)
        dzl[y, range(m)] -= 1
        wl, zl, al_1 = cache["l"+str(len(self.layers))]
        dwl = 1/m*np.dot(dzl, al_1.T)
        dbl = 1/m*np.sum(dzl, axis=1, keepdims=True)
        dal = np.dot(wl.T, dzl)
        #print(dbl)
        grads["dW"+str(len(self.layers))] = dwl
        grads["db"+str(len(self.layers))] = dbl
        
        for i in reversed(range(1, len(self.layers))):
            wl, zl, al_1 = cache["l"+str(i)]
            dal, dwl, dbl = backward_propagation(dal, wl, zl, al_1, self.layers[i-1].activation)
            grads["dW"+str(i)] = dwl
            grads["db"+str(i)] = dbl
        return grads
    
    def loss(self, y_hat, y):
        m = y.shape[-1]
        J = -1/m*np.sum(y*np.log(y_hat))
        return J
    
    def update(self, grads, learning_rate):
        for layer, i in zip(self.layers, range(1, len(self.layers)+1)):
            layer.W = layer.W - learning_rate*grads["dW"+str(i)]
            layer.b = layer.b - learning_rate*grads["db"+str(i)]
    
    def train(self, x, y, epoches, learning_rate, batch_size):
        losses = []
        for i in range(epoches):
            for X_batch, Y_batch in batchify(x, y, batch_size):
                y_onehot = one_hot(self.layers[-1].b.shape[0], Y_batch)
                y_hat, cache = self.forward_prop(X_batch)
                J = self.loss(y_hat, y_onehot)
                grads = self.backward_prop(y_hat, Y_batch, cache)
                self.update(grads, learning_rate)
            if i %(epoches/10) == 0:
                print(J)
            losses.append(J)
        return losses
    
    def evaluate(self, x, y):
        y_hat, _ = self.forward_prop(x)
        acc = (np.argmax(y_hat, axis=0) == y).astype("int")
        acc = np.sum(acc) / acc.shape[-1]
        return acc

In [2]:
data = load_digits()
x = data["data"].T
y = data["target"].reshape(1, x.shape[-1])

def split(X, Y, ratio, rand_state=1):
    X = X.T
    Y = Y.T
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ratio, random_state=rand_state)
    X_train = X_train.T
    X_test = X_test.T
    Y_train = Y_train.T
    Y_test = Y_test.T
    return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = split(x, y, 0.3)
print(X_train.shape, Y_train.shape)

(64, 1257) (1, 1257)


In [3]:
np.random.seed(1)
nets = model()
layers = [layer((128, 64), "sigmoid"), layer((10,128), "softmax")]
nets.layers = layers
losses = nets.train(X_train, Y_train, 10000, 0.1, 64)

2.084320072069936
0.001521597523028939
0.0006474677725406452
0.0004427932244254944
0.0003078609367274892
0.00023764599844507416
0.00021117331020050736
0.00016076341149558933
0.00015207183970701065
0.00014691034560256698


In [4]:
acc = nets.evaluate(X_test, Y_test)
print("test accuracy: "+str(acc*100)+"%")

test accuracy: 98.51851851851852%
